#chatbots



In [229]:
import numpy as np
import matplotlib.pyplot as plt
import spacy
from spacy import displacy
import nltk
import pandas as pd
from keras.preprocessing.sequence import pad_sequences

In [230]:
import pickle
with open('/content/train_qa.txt','rb') as f:
  train_data = pickle.load(f)

In [231]:
with open('/content/test_qa.txt','rb') as f:
  test_data = pickle.load(f)

In [232]:
all_data  = train_data + test_data

In [233]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [234]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [235]:
' '.join(train_data[0][2])

'n o'

In [236]:
df  = pd.DataFrame(all_data,columns=['Story','Question','Answer'])
df

,Story,Question,Answer
0,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Sandra, in, the, hallway, ?]",no
1,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bathroom, ?]",no
2,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, office, ?]",no
3,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bedroom, ?]",yes
4,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bedroom, ?]",yes
...,...,...,...
10995,"[Mary, moved, to, the, kitchen, ., Mary, trave...","[Is, Mary, in, the, bedroom, ?]",no
10996,"[Mary, moved, to, the, kitchen, ., Mary, trave...","[Is, Sandra, in, the, kitchen, ?]",no
10997,"[Mary, moved, to, the, kitchen, ., Mary, trave...","[Is, Mary, in, the, bedroom, ?]",no
10998,"[Mary, moved, to, the, kitchen, ., Mary, trave...","[Is, Sandra, in, the, garden, ?]",yes


In [237]:
vocab = set()
for story,question,answer in all_data:
  vocab = vocab.union(set(story))
  vocab = vocab.union(set(question))


In [238]:
vocab.add('yes')
vocab.add('no')
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [239]:
vocab_len = len(vocab) + 1
# for i in range(len(all_data)):
#   for data in all_data[i]:
#     print(' '.join(data))

In [240]:
stories_len = [len(data[0]) for data in all_data]
question_len = [len(data[1]) for data in all_data]

In [241]:
maxlen_stories = max(stories_len)
maxlen_stories

156

In [242]:
maxlen_question = max(question_len)
maxlen_question

6

In [243]:
tokenizer = Tokenizer(filters = [])

In [244]:
tokenizer.fit_on_texts(vocab)

In [245]:
tokenizer.word_index['yes']

12

In [246]:
tokenizer.word_index['no']

29

In [247]:
tokenizer.word_index

{'travelled': 1,
 'apple': 2,
 '?': 3,
 'put': 4,
 'moved': 5,
 'discarded': 6,
 'office': 7,
 'john': 8,
 'bathroom': 9,
 'there': 10,
 'to': 11,
 'yes': 12,
 'left': 13,
 'journeyed': 14,
 'is': 15,
 '.': 16,
 'down': 17,
 'football': 18,
 'milk': 19,
 'garden': 20,
 'took': 21,
 'got': 22,
 'mary': 23,
 'hallway': 24,
 'went': 25,
 'in': 26,
 'kitchen': 27,
 'grabbed': 28,
 'no': 29,
 'dropped': 30,
 'picked': 31,
 'daniel': 32,
 'sandra': 33,
 'back': 34,
 'up': 35,
 'the': 36,
 'bedroom': 37}

In [248]:
train_story_text = []
train_question_text = []
train_answer_text = []
for s,q,a in train_data:
  train_story_text.append(s)
  train_question_text.append(q)
  train_answer_text.append(q)

In [249]:
# train_story_text

In [250]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)
# train_story_seq = np.array(train_story_seq)
# train_story_seq

In [251]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=maxlen_stories,max_question_len=maxlen_question):
    '''
    OUTPUT:
    Vectorizes the stories,questions, and answers into padded sequences. We first loop for every story, query , and
    answer in the data. Then we convert the raw words to an word index value. Then we append each set to their appropriate
    output list. Then once we have converted the words to numbers, we pad the sequences so they are all of equal length.
    Returns this in the form of a tuple (X,Xq,Y) (padded based on max lengths)
    '''
    # X = STORIES
    X = []
    # Xq = QUERY/QUESTION
    Xq = []
    # Y = CORRECT ANSWER
    Y = []


    for story, query, answer in data:

        # Grab the word index for every word in story
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]
        y = np.zeros(len(word_index) + 1)
        y[word_index[answer]] = 1

        X.append(x)
        Xq.append(xq)
        Y.append(y)

    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [252]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [253]:
import keras.layers
from keras.models import Sequential, Model
from keras.layers import Embedding,Input, Activation, Dense, Permute, Dropout,add, dot, concatenate,LSTM

In [254]:
input_sequence = Input((maxlen_stories,))
question_sequence = Input((maxlen_question,))
question_sequence

<KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'input_10')>

### input encoder M

In [255]:
# Input gets embedded to a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_len,output_dim=64))
input_encoder_m.add(Dropout(0.5))

# This encoder will output:
# (samples, story_maxlen, embedding_dim)

### input encoder C

In [256]:
# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_len,output_dim=maxlen_question))
input_encoder_c.add(Dropout(0.5))
# output: (samples, story_maxlen, query_maxlen)

### question encoder


In [257]:
# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_len,
                               output_dim=64,
                               input_length=maxlen_question))
question_encoder.add(Dropout(0.5))
# output: (samples, query_maxlen, embedding_dim)

In [258]:
# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question_sequence)

In [259]:
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

In [260]:
# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

In [261]:
# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])
answer

<KerasTensor: shape=(None, 6, 220) dtype=float32 (created by layer 'concatenate_3')>

In [262]:
# Reduce with RNN (LSTM)
answer = LSTM(32)(answer)  # (samples, 32)
# Regularization with Dropout
answer = Dropout(0.5)(answer)
answer = Dense(vocab_len)(answer)  # (samples, vocab_size)
# probability distribution over the vocabulary
answer = Activation('softmax')(answer)

In [263]:
model = Model([input_sequence, question_sequence], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [264]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 156)]                0         []                            
                                                                                                  
 input_10 (InputLayer)       [(None, 6)]                  0         []                            
                                                                                                  
 sequential_12 (Sequential)  (None, None, 64)             2432      ['input_9[0][0]']             
                                                                                                  
 sequential_14 (Sequential)  (None, 6, 64)                2432      ['input_10[0][0]']            
                                                                                            

In [265]:
model.fit([inputs_train, queries_train], answers_train,batch_size=32,epochs=100,validation_data=([inputs_test, queries_test], answers_test))

Epoch 1/100
313/313 [==============================] - 5s 9ms/step - loss: 0.8663 - accuracy: 0.4928 - val_loss: 0.6941 - val_accuracy: 0.5030
Epoch 2/100
313/313 [==============================] - 2s 6ms/step - loss: 0.7035 - accuracy: 0.5005 - val_loss: 0.6934 - val_accuracy: 0.4970
Epoch 3/100
313/313 [==============================] - 2s 6ms/step - loss: 0.6969 - accuracy: 0.4992 - val_loss: 0.6935 - val_accuracy: 0.4970
Epoch 4/100
313/313 [==============================] - 2s 7ms/step - loss: 0.6971 - accuracy: 0.4894 - val_loss: 0.6933 - val_accuracy: 0.5030
Epoch 5/100
313/313 [==============================] - 3s 9ms/step - loss: 0.6957 - accuracy: 0.4986 - val_loss: 0.6945 - val_accuracy: 0.4970
Epoch 6/100
313/313 [==============================] - 2s 6ms/step - loss: 0.6956 - accuracy: 0.4967 - val_loss: 0.6948 - val_accuracy: 0.4970
Epoch 7/100
313/313 [==============================] - 2s 7ms/step - loss: 0.6954 - accuracy: 0.4975 - val_loss: 0.6934 - val_accuracy: 0.5030

In [266]:
filename = 'chatbot.h5'
model.save(filename)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [267]:
model.load_weights(filename)
pred_results = model.predict(([inputs_test, queries_test]))

32/32 [==============================] - 0s 2ms/step


In [268]:
story =' '.join(word for word in test_data[1][0])
print(story)

Mary got the milk there . John moved to the bedroom . Mary discarded the milk . John went to the garden .


In [269]:
query = ' '.join(word for word in test_data[1][1])
print(query)

Is John in the kitchen ?


In [270]:
print("Answer is:",test_data[1][2])

Answer is: no


In [271]:
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  no
Probability of certainty was:  0.99700576


In [272]:
my_story = "John left the kitchen . Sandra dropped the football in the garden ."
my_story.split()

['John',
 'left',
 'the',
 'kitchen',
 '.',
 'Sandra',
 'dropped',
 'the',
 'football',
 'in',
 'the',
 'garden',
 '.']

In [273]:
my_question = "is sandra in the kitchen ?"
my_question.split()

['is', 'sandra', 'in', 'the', 'kitchen', '?']

In [274]:
mydata = [(my_story.split(),my_question.split(),)]

In [275]:
my_story,my_ques,my_ans = vectorize_stories(mydata)

In [276]:
pred_results = model.predict(([ my_story, my_ques]))
pred_results

1/1 [==============================] - 0s 19ms/step


array([[5.5883888e-08, 5.1987968e-08, 3.6268123e-08, 4.9672881e-08,
        6.5013417e-08, 4.9620606e-08, 5.1467431e-08, 5.7577164e-08,
        5.5401742e-08, 4.7160668e-08, 5.4266479e-08, 4.1170370e-08,
        1.2862320e-02, 6.0764677e-08, 4.6846345e-08, 4.8501114e-08,
        5.9824323e-08, 3.4170192e-08, 5.8537893e-08, 4.7336492e-08,
        5.1248204e-08, 4.9300713e-08, 4.9333444e-08, 6.0395507e-08,
        5.7711741e-08, 5.6791873e-08, 4.7608673e-08, 4.8058183e-08,
        5.8120246e-08, 9.8713595e-01, 5.1824120e-08, 5.6403248e-08,
        4.4772772e-08, 5.2171441e-08, 4.7040107e-08, 4.4247781e-08,
        4.1225530e-08, 5.3670075e-08]], dtype=float32)

In [277]:
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  no
Probability of certainty was:  0.98713595
